 ## Export ONNX model

In [2]:
from ultralytics import YOLO

model = YOLO("../saves/yolo11m_osrs_21c.pt") # load with our trained weights

model.export(format="onnx",imgsz=320, device='cpu', nms=True) # input shape (1x3x320x320), target cpu, non-max sup attached

Ultralytics 8.3.140 🚀 Python-3.9.21 torch-2.6.0+cu118 CPU (AMD Ryzen 9 5900X 12-Core Processor)
YOLO11m summary (fused): 125 layers, 20,046,223 parameters, 0 gradients, 67.7 GFLOPs

PyTorch: starting from '../saves/yolo11m_osrs_21c.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 300, 6) (38.6 MB)
requirements: Ultralytics requirements ['onnxslim>=0.1.53', 'onnxruntime'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.8s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.52...
ONNX: export success ✅ 4.9s, saved as '../saves/yolo11m_osrs_21c.onnx' (76.6 MB)

Export complete (5.2s)
Results saved to /home/chitraz/Documents/Projects/RealTimeODinOSRS/saves
Predict:         yolo predict task=detect model=../saves/yolo11m_osrs_21c.onnx imgsz=320  
Validate:        yolo val task=detect model=../saves/yolo11m_osrs_21c.onnx imgsz=320 

'../saves/yolo11m_osrs_21c.onnx'

## Load ONNX model and run inference using onnxruntime

In [9]:
import onnxruntime 
import numpy as np
import cv2 as cv
import os

# onnx session
ort_sess = onnxruntime.InferenceSession('../saves/yolo11m_osrs_21c.onnx')

# create/pre-process a input batch 
root_path = os.path.join('/home/chitraz/Documents/Projects/RealTimeODinOSRS', 'dataset/val/images')
img_path = root_path + "/2922_3287_0_980_154_1945.png" # test image
input_img = cv.imread(img_path)
input_img = cv.resize(input_img, dsize=(320, 320), interpolation=cv.INTER_CUBIC) # resize to 320x320
input_img = np.transpose(input_img,(2,0,1)) # (HxWxC) -> (CxHxW) 
input_img = input_img.astype(np.float32)
input_img = np.expand_dims(input_img, 0) # add fake mini-batch bim

# inference 
outputs = ort_sess.run(None,{"images": input_img})[0][0]
outputs

array([[     49.507,      62.007,      128.78,      104.65,           1,          18],
       [     251.42,      188.01,      276.09,      219.93,           1,          18],
       [     252.09,      187.35,      284.32,      229.02,           1,          18],
       ...,
       [     134.84,      268.15,      225.48,      301.32,     0.41343,           2],
       [     2.6918,      -2.084,        36.9,      16.623,     0.41072,           2],
       [      89.19,      155.82,      186.99,      192.11,     0.40337,          18]], dtype=float32)

In [7]:
class_filter = ['Cow','Cow_calf', 'Dairy_cow'
    ,'Tree','Evergreen_tree','Oak_tree','Willow_tree','Yew_tree'
    ,'Plant','Daisies','Thistle'
    ,'Rocks','Copper_rocks','Tin_rocks','Silver_rocks','Iron_rocks','Coal_rocks', 'Gold_rocks', 'Mithril_rocks', 'Adamantite_rocks', 'Clay_rocks']

CONF_TH = 0

for pred in outputs:
    box = pred[:3]
    conf = pred[4]
    class_id = pred[5]
    class_label = class_filter[int(class_id)]
    if conf > CONF_TH:
        print(pred)

[     49.507      62.007      128.78      104.65           1          18]
[     251.42      188.01      276.09      219.93           1          18]
[     252.09      187.35      284.32      229.02           1          18]
[        252      195.76      284.05         236           1          18]
[     261.36      188.79       284.4         236           1           4]
[     236.64      210.03      268.11      243.66           1          18]
[     245.52      204.02       275.9      243.87           1          18]
[     253.28      204.07      284.54         241           1          18]
[      250.7      213.95       278.4      251.97           1          18]
[     251.97       219.8      283.99      251.99           1          18]
[     281.57      211.99      324.69      251.99           1          18]
[     289.92         220      319.36      259.69           1          18]
[     293.69      229.52      319.87      262.58           1          18]
[     60.679      59.963      118.43  

In [1]:
# clean up multi boxes for same object (NMS)

def nms(boxes, confs, IOU_TH  = 0.6):

    pass